In [1]:
!pip install feedparser # RSS에서 xml태그별 정보추출(예: title, link, ..)
!pip install newspaper3k # 인터넷 신문기사 분석(아래 'Article()'를 사용하기 위함)
!pip install konlpy # 한국어 형태소 분석기(주어진 문장에서 명사만 추출)

import feedparser # !pip install feedparser
from newspaper import Article # !pip install newspaper3k
from konlpy.tag import Okt # !pip install konlpy
from collections import Counter # 명사추출후에 본문에 몇번나오는지 확인(TF 계산용)
from operator import eq # 동일한 단어인지 여부를 판단
from bs4 import BeautifulSoup # 글에 존재할지 모르는 html태그 삭제

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=8efee6af73efccd3f5e9700280872b7d3d307be543b3db74648915770dbaf350
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=4e3f344c58f013f25c4a3019f6c9f52fe83abaa270c05abde76d4eacc9edb34f
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58d

In [2]:
# 단계1: 모든 RSS파일(xml형식)을 돌아다니며 기사의 제목/link를 추출함
# urls 는 우리가 검색할 RSS의 목록을 list로 만든 것
urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml",
        ]

# 아래 함수는 RSS목록의 list 안에 존재하는 모든 기사의 title, link를 list로 구성
def crawl_rss(urls):
    array_rss = [] # 함수 시작하는 시점에 빈 리스트를 만듦. 여기에 모든 기사 채울것임
    titles_rss = set() # 중복기사제거: 기사제목들의 집합을 구성(집합은 중복 불허)
    for url in urls: # 4개의 RSS파일을 하나씩 방문(4번 돌것임)
        print("[Crawl RSS]",url) # 현재 어디에 있는지 출력
        parse_rss = feedparser.parse(url) # 현재 url를 파싱해서 그 결과를 parse_rss에 저장
        for p in parse_rss.entries:  # parse_rss에 있는 모든 entires/기사를 검색
            if p.title not in titles_rss: # 중복기사제거 : 만약 titles_rss에 동일한 제목이 없다면 추가
                array_rss.append({'title':p.title, 'link':p.link}) # 기사에서 제목/link 추출후 리스트에 추가
                titles_rss.add(p.title) # 중복기사제거 : 집합에 현재 기사제목이 없을 때만 추가
            else:
                print('Duplicated Title:',p.title) # 중복기사제거 : 중복되는 기사의 제목출력
    return array_rss

list_articles = crawl_rss(urls)
print(list_articles)

[Crawl RSS] http://rss.etnews.com/Section901.xml
[Crawl RSS] http://rss.etnews.com/Section902.xml
Duplicated Title: 빗썸 메타버스 출격준비 완료…16조 시장 '패션·데이팅' 잡는다
[Crawl RSS] http://rss.etnews.com/Section903.xml
Duplicated Title: 아이폰15 시리즈, 품귀현상 나타날 것으로 전망
Duplicated Title: 칼부림부터 학교폭력 대응까지…다재다능한 지능형 CCTV 주목
Duplicated Title: 토요타 HEV 마지막 퍼즐 '신형 프리우스' 연말 韓 상륙
[Crawl RSS] http://rss.etnews.com/Section904.xml
Duplicated Title: 빗썸 메타버스 출격준비 완료…16조 시장 '패션·데이팅' 잡는다
Duplicated Title: 아이파트너즈, 혁신 스타트업 유럽진출 지원 시동
Duplicated Title: 롯데칠성음료, 필리핀펩시 경영권 취득...글로벌 음료시장 공략
Duplicated Title: [미래 반도체 유니콘 열전]〈4〉퀄리타스반도체, 대규모 R&D로 '글로벌 인터커넥트 IP 기업' 도전
Duplicated Title: 사공민 GS리테일 점포운영지원실장 “중기·점주 협업, 친환경 편의점 시대 연다”
Duplicated Title: 제주삼다수, 먹는샘물 25년간 부동의 1위 비결은
Duplicated Title: [단독] 수자원공사 해외투자 난관…'원금 회수' 적신호
Duplicated Title: 산업부, '무역·기술 안보 종합전략' 세운다…연내 포럼 구성
Duplicated Title: '100경분의 1초' 빛으로 '전자의 세계' 연 과학자들 노벨 물리학상 수상
Duplicated Title: 정부, 디플정 마중물 '전자정부 클라우드 플랫폼' 구축 착수
Duplicated Title: [에듀플러스]“특성화고, 신산업·신기술 분야에 맞춘 학과 개편

In [3]:
# 단계2 : list에 존재하는 모든 링크를 돌아다니며 본문 text를 추출

# 아래 함수는 하나의 url을 입력받아서, 링크를 타고 들어가, 그 안에 title과 text를 추출한다.
# 디폴트로 한글로 지정한다.
def crawl_article(url, language='ko'):
    print("[Crawl Article]",url) # 현재 title과 text를 추출한 url을 프린트
    a = Article(url, language=language) # Article을 사용하여 그 URL입력하고, 언어옵션지정, a에 저장
    a.download() # a에 해당하는 url기사 다운로드
    a.parse() # a에 해당하는 url 기사 분석
    return a.title, preprocessing(a.text) # a에 해당하는 title과 본문 출력

def preprocessing(text):
    text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그 제거
    return text_article



for article in list_articles: # list에 있는 모든 기사를 하나씩 방문
    _, text = crawl_article(article['link']) # 그 기사의 link를 crawl_article함수에 넣어 본문 추출
    article['text'] = text # 추출된 본문을 list_articles에 'text'라는 속성 새로 만들어 저장
print(list_articles[0]) # 첫번째 기사를 출력(title, link, text가 모두 나오는 것 확인)

[Crawl Article] https://www.etnews.com/20231004000087
[Crawl Article] https://www.etnews.com/20231004000053
[Crawl Article] https://www.etnews.com/20231004000021
[Crawl Article] https://www.etnews.com/20231003000006
[Crawl Article] https://www.etnews.com/20230927000112
[Crawl Article] https://www.etnews.com/20231004000004
[Crawl Article] https://www.etnews.com/20231004000003
[Crawl Article] https://www.etnews.com/20231004000002
[Crawl Article] https://www.etnews.com/20231003000103
[Crawl Article] https://www.etnews.com/20230926000252
[Crawl Article] https://www.etnews.com/20231002000054
[Crawl Article] https://www.etnews.com/20231002000053
[Crawl Article] https://www.etnews.com/20230926000326
[Crawl Article] https://www.etnews.com/20230926000306
[Crawl Article] https://www.etnews.com/20230926000298
[Crawl Article] https://www.etnews.com/20231003000057
[Crawl Article] https://www.etnews.com/20231003000019
[Crawl Article] https://www.etnews.com/20231003000017
[Crawl Article] https://www.

In [26]:
len(list_articles)

76

In [29]:
# 단계3 : 모든 본문text에서 명사(키워드, 빈도수) 추출
from math import log
def get_keywords(text, nkeywords=10): # 키워드 추출 함수, 디폴트로 10개 지정
    spliter = Okt() # konlpy에 의해서 문장을 형태소별로 쪼개는 기능을 위해 spliter 생성
    nouns = spliter.nouns(text) # spliter 에 의해서 nouns함수를 불러 text를 넣으면 그 text의 명사만 추출
    count = Counter(nouns) # 추출된 명사들의 출현빈도 추출
    list_keywords = [] # 비어있는 키워드 리스트를 먼저 만듦
    for n, c in count.most_common(nkeywords): # 가장 출현빈도 높은 명사부터 순차적으로 10번 출력
        item = {'keyword':n, 'count':c} # 리스트에 저장은 {'keyword', 'count'} 형식으로 저장됨
        list_keywords.append(item) # 이런 저장된 포맷으로 이를 list_keywords에 붙여 나감
    return list_keywords

def get_idf(keyword, lst):
    count = 0
    for i in lst:
        if keyword in i['text']:
            count+=1
    print(len(lst), 1+count, len(lst)/(1+count) ,log(len(lst)/(1+count)))
    return log(len(lst)/(1+count))

for article in list_articles: # 모든 기사를 돌아다니면 text에서 명사추출, 키워드/빈도 추출
    keywords = get_keywords(article['text']) # get_keywords 함수로 키워드/빈도 추출
    article['keywords']=keywords # 추출된 키워드/빈도를 list_articles의 'keywords'로 저장

a=0
for article in list_articles:
    for i in range(len(article['keywords'])):
        idf = get_idf(article['keywords'][i]['keyword'], list_articles)
        article['keywords'][i]['idf'] = idf
        article['keywords'][i]['tfidf'] = article['keywords'][i]['count'] * article['keywords'][i]['idf']
        a+=1
print(a)
print(list_articles[0]) # 첫번째 기사 출력(title, link, text, keywords)

76 4 19.0 2.9444389791664403
76 4 19.0 2.9444389791664403
76 2 38.0 3.6375861597263857
76 9 8.444444444444445 2.133508762950112
76 2 38.0 3.6375861597263857
76 15 5.066666666666666 1.622683139184121
76 4 19.0 2.9444389791664403
76 71 1.0704225352112675 0.06805346324501557
76 2 38.0 3.6375861597263857
76 23 3.3043478260869565 1.1952391243571814
76 8 9.5 2.2512917986064953
76 5 15.2 2.7212954278522306
76 12 6.333333333333333 1.8458266904983307
76 22 3.4545454545454546 1.2396908869280152
76 10 7.6 2.028148247292285
76 12 6.333333333333333 1.8458266904983307
76 4 19.0 2.9444389791664403
76 16 4.75 1.55814461804655
76 12 6.333333333333333 1.8458266904983307
76 7 10.857142857142858 2.3848231912310176
76 5 15.2 2.7212954278522306
76 3 25.333333333333332 3.2321210516182215
76 2 38.0 3.6375861597263857
76 3 25.333333333333332 3.2321210516182215
76 12 6.333333333333333 1.8458266904983307
76 3 25.333333333333332 3.2321210516182215
76 66 1.1515151515151516 0.1410785982599056
76 6 12.66666666666666

In [30]:
print(list_articles[-1])

{'title': '[인터뷰]마이클 비셔 미국 노터데임대 교수, “중이온가속기 인상적...향후 안정된 예산확보로 잘 운영되길”', 'link': 'https://www.etnews.com/20230927000060', 'text': "“한국의 중이온가속기는 세계 다른 것과 비교해서도 상당히 인상적인 시설입니다. 이런 연구시설을 잘 유지해 이룬 기초과학 역량은 과학기술과 산업 전반의 발전 토대가 됩니다.”\n\n\n\n핵천체물리학 분야 세계적인 석학 마이클 비셔 미국 노터데임대 특훈 교수는 전자신문과 가진 인터뷰에서 우리나라의 중이온가속기(라온), 나아가 기초과학에 대한 투자와 지원이 향후 큰 보답을 부를 것이라고 강조했다.\n\n\n\n마이클 비셔 교수는 지난달 기초과학연구원(IBS) 희귀핵연구단, 중이온가속기연구소가 공동 개최한 천체물리 최대 규모 국제학회 '제17회 우주속 원자핵 국제심포지엄' 참가차 우리나라를 찾았다.\n\n\n\n핵천체물리학, 저에너지 핵물리 실험, 다양한 핵물리 분야를 연구하는 석학이다. 그가 몸담은 노터데임대에 가속기도 있어 해당 분야에 밝다.\n\n\n\n마이클 비셔 교수는 향후 우리 중이온가속기 위상과 역할이 상당할 것이라고 말했다.\n\n\n\n그는 “가능성이 보이는 시설로, 해외 가속기와 연구 협력해 큰 시너지를 낼 것”이라며 “특히 우주 원소의 기원 연구에 활용할 수 있고, 중성미자 연구에도 활약할 수 있을 것”이라고 전했다.\n\n기초과학 외 영역도 수혜를 입을 것이라고 강조했다. 그는 “이런 도전적인 연구는 최신 전기·전자 장비나 관련 인공지능(AI) 등 소프트웨어(SW) 개발도 견인하게 돼, 과학기술과 산업 전반의 발전도 이룰 수 있게 한다”며 “재료공학, 컴퓨터 시뮬레이션, 의학분야 등에 직접 활용도도 높다”고 강조했다.\n\n\n\n세계 곳곳의 연구자들도 중이온가속기를 주목하고 있고, 또 협력을 바랄 것이라는 견해도 밝혔다.\n\n\n\n마이클 피셔 교수는 또 “가속기는 전세계에 많지 않고, 한 시설이 한 번에 여러

In [33]:
# 단계4 : 검색어를 입력받아서 그 검색어를 가지고 있는 기사를 출력
query = input()

def search_articles(query, list_keywords): # 쿼리가 키워드리스트에 있으면 회수를 출력(없으면 0)
    nWords, idf, tf_idf = 0, 0, 0 # 아래의 if문에 걸리지 않으면(즉 쿼리가 키워드에 없으면) 0을 출력하기 위함

    for kw in list_keywords: # 키워드 리스트 검색
        if eq(query, kw['keyword']): # 쿼리와 동일한 키워드가 존재한다면 ## query == kw['keyword'] 도 가능
            nWords, idf, tf_idf = kw['count'], kw['idf'], kw['tfidf']# 그러면 그 키워드에 해당하는 count가 nWords값이 됨
    return nWords, idf, tf_idf # 결과적으로 쿼리의 출현횟수를 출력

lst = []
for article in list_articles: # 모든 기사를 돌아다니며 입력받은 쿼리가 몇 번 나오나 검색
    nQuery,idf, tf_idf = search_articles(query, article['keywords'])
    if nQuery != 0: # 쿼리가 키워드에 존재하지 않는 경우는 출력하지 않음
        print('[TF]',nQuery, '[IDF]',idf,'[TF-IDF]',tf_idf, '[title]',article['title'], '[URL]', article['link'])
        lst.append([nQuery, idf, tf_idf, article['title'], article['link']])
print("="*100)
print("정렬 후")
print("="*100)
#sorted(lst, key=lambda x: x[0])
for i in sorted(lst, key=lambda x: x[0], reverse=True):
    print('[TF]',i[0], '[IDF]',i[1],'[TF-IDF]',i[2],'[title]',i[3], '[URL]', i[4])

중이온
[TF] 7 [IDF] 3.6375861597263857 [TF-IDF] 25.4631031180847 [title] [인터뷰]마이클 비셔 미국 노터데임대 교수, “중이온가속기 인상적...향후 안정된 예산확보로 잘 운영되길” [URL] https://www.etnews.com/20230927000060
정렬 후
[TF] 7 [IDF] 3.6375861597263857 [TF-IDF] 25.4631031180847 [title] [인터뷰]마이클 비셔 미국 노터데임대 교수, “중이온가속기 인상적...향후 안정된 예산확보로 잘 운영되길” [URL] https://www.etnews.com/20230927000060


In [34]:
query = input()

def search_articles(query, list_keywords): # 쿼리가 키워드리스트에 있으면 회수를 출력(없으면 0)
    nWords, idf, tf_idf = 0, 0, 0 # 아래의 if문에 걸리지 않으면(즉 쿼리가 키워드에 없으면) 0을 출력하기 위함

    for kw in list_keywords: # 키워드 리스트 검색
        if eq(query, kw['keyword']): # 쿼리와 동일한 키워드가 존재한다면 ## query == kw['keyword'] 도 가능
            nWords, idf, tf_idf = kw['count'], kw['idf'], kw['tfidf']# 그러면 그 키워드에 해당하는 count가 nWords값이 됨
    return nWords, idf, tf_idf # 결과적으로 쿼리의 출현횟수를 출력

lst = []
for article in list_articles: # 모든 기사를 돌아다니며 입력받은 쿼리가 몇 번 나오나 검색
    nQuery,idf, tf_idf = search_articles(query, article['keywords'])
    if nQuery != 0: # 쿼리가 키워드에 존재하지 않는 경우는 출력하지 않음
        print('[TF]',nQuery, '[IDF]',idf,'[TF-IDF]',tf_idf, '[title]',article['title'], '[URL]', article['link'])
        #lst.append([nQuery, idf, tf_idf, article['title'], article['link']])

중이온
[TF] 7 [IDF] 3.6375861597263857 [TF-IDF] 25.4631031180847 [title] [인터뷰]마이클 비셔 미국 노터데임대 교수, “중이온가속기 인상적...향후 안정된 예산확보로 잘 운영되길” [URL] https://www.etnews.com/20230927000060


In [41]:
i=0
for i, article in enumerate(list_articles):
    print(article['keywords'])

[{'keyword': '메타', 'count': 18, 'idf': 2.9444389791664403, 'tfidf': 52.99990162499593}, {'keyword': '버스', 'count': 10, 'idf': 2.9444389791664403, 'tfidf': 29.444389791664403}, {'keyword': '빗썸', 'count': 8, 'idf': 3.6375861597263857, 'tfidf': 29.100689277811085}, {'keyword': '플랫폼', 'count': 7, 'idf': 2.133508762950112, 'tfidf': 14.934561340650784}, {'keyword': '패션', 'count': 6, 'idf': 3.6375861597263857, 'tfidf': 21.825516958358314}, {'keyword': '서비스', 'count': 6, 'idf': 1.622683139184121, 'tfidf': 9.736098835104727}, {'keyword': '상품', 'count': 6, 'idf': 2.9444389791664403, 'tfidf': 17.66663387499864}, {'keyword': '수', 'count': 6, 'idf': 0.06805346324501557, 'tfidf': 0.40832077947009343}, {'keyword': '커머스', 'count': 5, 'idf': 3.6375861597263857, 'tfidf': 18.18793079863193}, {'keyword': '시장', 'count': 5, 'idf': 1.1952391243571814, 'tfidf': 5.976195621785907}]
[{'keyword': '유럽', 'count': 12, 'idf': 2.2512917986064953, 'tfidf': 27.015501583277945}, {'keyword': '스타트업', 'count': 11, 'idf': 2